In [1]:
import KeyFunctions as kf
import pandas as pd
import numpy as np

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
#Import the postcode list from the raw data gdf
# gdf = kf.load_obj(root_path,"raw_gdf")
name = "M25" 
gdf = kf.load_obj(root_path,"raw_gdf_" + name)

In [3]:
#gdf["Type"].unique().tolist()

['RailwayStations',
 'BusStops',
 'RoadAccidents',
 'Postcodes',
 'UDPRNs',
 'LReg',
 'Schools',
 'Crimes',
 'StopAndSearch',
 'Clinic',
 'Dentists',
 'GP',
 'Opticians',
 'Pharmacy',
 'Care homes and care at home',
 'All_GB',
 'LocalAuthorities',
 'LSOA',
 'GreenSpace',
 'Rivers',
 'Railway_Lines',
 'Urban_Region',
 'Lakes',
 'Road',
 'MotorwayJunction',
 'SiteOfScientificInterest',
 'LocalNatureReserve',
 'AncientWoodland',
 'ConservationArea',
 'Listed Buildings',
 'MetOffice']

In [4]:
#Shapes
LSOA = gdf.loc[gdf["Type"]=="LSOA",:].reset_index(drop=True)

#Points
LReg = gdf.loc[gdf["Type"]=="LReg",:].reset_index(drop=True)
Crimes = gdf.loc[gdf["Type"]=="Crimes",:].reset_index(drop=True)
UDPRNs = gdf.loc[gdf["Type"]=="UDPRNs",:].reset_index(drop=True)

#Add Sales and Average Cost for each LSOA
LSOA["LReg_Sales_In_LSOA"] = [np.count_nonzero(LReg.loc[LReg.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry]
LSOA["LReg_AvCost_In_LSOA"] = [np.sum(LReg.loc[LReg.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry]/LSOA["LReg_Sales_In_LSOA"]
LSOA["UDPRNs"] = [np.count_nonzero(UDPRNs.loc[UDPRNs.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry]

#Add crimes for each LSOA
Crime_Types = set(gdf.loc[gdf["Type"]=="Crimes",:].loc[:,"Name"].to_list())
Crime_List = []

for c in Crime_Types:
    new_name = "Crimes_per1000houses_"+c.replace(' ', '_')
    sub_Crimes = Crimes.loc[Crimes["Name"]==c,:]
    LSOA[new_name] = ([np.count_nonzero(sub_Crimes.loc[sub_Crimes.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry] / LSOA["UDPRNs"])*1000
    Crime_List = Crime_List + [new_name]

#Create LSOA Network
network, pos = kf.create_network_from_shapes(LSOA)

#Calculate densities
LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "LReg_Sales_In_LSOA", "LReg_AvCost_In_LSOA", 1).iloc[:,1]], axis=1)
LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "LReg_Sales_In_LSOA", "LReg_AvCost_In_LSOA", 3).iloc[:,1]], axis=1)
LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "LReg_Sales_In_LSOA", "LReg_AvCost_In_LSOA", 5).iloc[:,1]], axis=1)

for c in Crime_List:
    LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "UDPRNs", c, 1).iloc[:,1]], axis=1)
    LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "UDPRNs", c, 2).iloc[:,1]], axis=1)
    LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "UDPRNs", c, 5).iloc[:,1]], axis=1)

LSOA = LSOA.drop(columns=['Type', 'Details_Str', 'Details_Float', 'geometry'])

del LReg, Crimes, UDPRNs

In [5]:
#get nspl data
NSPL_infile = root_path + "NSPL_NOV_2020_UK/Data/NSPL_NOV_2020_UK.csv"

NSPL = pd.read_csv(NSPL_infile, low_memory=False)

LSOA_Lookup_path = root_path + "NSPL_NOV_2020_UK/Documents/LSOA (2011) names and codes UK as at 12_12.csv"

LSOA_Lookup = pd.read_csv(LSOA_Lookup_path)

In [ ]:
#Combine the NSPL data with it's lookups for the variables we want
NSPL = NSPL.loc[:,["pcd", "lsoa11"]]
NSPL["pcd"] = NSPL["pcd"].astype(str).str.replace(" ","")
NSPL = NSPL.merge(LSOA_Lookup, left_on="lsoa11", right_on="LSOA11CD")
NSPL = NSPL.drop(["lsoa11", "LSOA11CD"], axis=1)
NSPL = NSPL.rename(columns={"pcd": "Postcode"})
NSPL = NSPL.rename(columns={"LSOA11NM": "LSOA"})

In [ ]:
#Initialise our postcodes gdf as a list of postcodes and their points
Postcodes = gdf.loc[gdf["Type"]=="Postcodes",["Name", "geometry"]].copy()
Postcodes = Postcodes.rename(columns={"Name": "Postcode"})

Postcodes = Postcodes.set_index("Postcode")

#Add on nspl data
Postcodes = Postcodes.merge(NSPL, how="left", left_on="Postcode", right_on="Postcode")
Postcodes = Postcodes.merge(LSOA, left_on="LSOA", right_on="Name",how="inner")

In [ ]:
#List out the points we will want to use for densities, point distances...etc.

All_Crimes = gdf.loc[gdf["Type"]=="Crimes",:]
All_Schools = gdf.loc[gdf["Type"]=="Schools",:]
Hospitals = gdf.loc[gdf["Type"]=="Hospital",:]
RailwayStations = gdf.loc[gdf["Type"]=="RailwayStations",:]
FerryTerminals = gdf.loc[gdf["Type"]=="FerryTerminals",:]
BusStops = gdf.loc[gdf["Type"]=="BusStops",:]
ListedBuildings = gdf.loc[gdf["Type"]=="Listed Buildings",:]
PropertySales = gdf.loc[gdf["Type"]=="LReg",:]
RoadAccidents = gdf.loc[gdf["Type"]=="RoadAccidents", :]
StopAndSearch = gdf.loc[gdf["Type"]=="StopAndSearch", :]
FireStations = gdf.loc[gdf["Type"]=="FireStations", :]
roads = gdf.loc[gdf["Type"]=='Road', :]
A_Road = roads.loc[roads["Name"]=='A Road']
Motorway = roads.loc[roads["Name"]=='Motorway']

TotalRainfall_mm_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='TotalRainfall_mm_2020'),:]
Snow_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Snow_Days_2020'),:]
Sunlight_h_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Sunlight_h_2020'),:]
AverageTemperature_C_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='AverageTemperature_C_2020'),:]
GroundFrost_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='GroundFrost_Days_2020'),:]
#...etc.

#Point datasets to find the density of and the radius of those densities in the format output_name:dataframe where the output variable will have the name Outname_Density_Radius
density_var_dict = {
    "Postcodes":Postcodes, 
    "All_Crimes":All_Crimes, 
    "All_Schools":All_Schools, 
    "Hospitals":Hospitals, 
    "RailwayStations":RailwayStations,
    "FerryTerminals":FerryTerminals,
    "BusStops":BusStops,
    "ListedBuildings":ListedBuildings,
    "RoadAccidents":RoadAccidents,
    "StopAndSearchs":StopAndSearch,
    "FireStations":FireStations
}

#density_r_list = [1000,5000,25000]
density_r_list = [1000]

#Point datasets to find the nearest of in the format output_name:dataframe where the output variable will have the name Nearest_Out_Name
nearest_var_dict = {
    "School":All_Schools, 
    "Hospital":Hospitals, 
    "RailwayStation":RailwayStations,
    "FerryTerminals":FerryTerminals,
    "BusStops":BusStops,
    "ListedBuilding":ListedBuildings,
    "A_Road":A_Road,
    "Motorway":Motorway
}

#Points to find the average of
average_var_dict = {
    "ProertyPrice_2020_2021":[PropertySales, "Details_Float"],
    "TotalRainfall_mm_2020":[TotalRainfall_mm_2020, "Details_Float"],
    "Snow_Days_2020":[Snow_Days_2020, "Details_Float"],
    "Sunlight_h_2020":[Sunlight_h_2020, "Details_Float"],
    "AverageTemperature_C_2020":[AverageTemperature_C_2020, "Details_Float"],
    "GroundFrost_Days_2020":[GroundFrost_Days_2020, "Details_Float"]
}

In [ ]:
#Start with adding density columns
for k in density_var_dict:
    print("Variable: " + k)
    for r in density_r_list:
        print("Density: " + str(r))
        Postcodes[k + "_Density_" + str(r)] = kf.within_radius(Postcodes,density_var_dict[k], r)

Variable: Postcodes
Density: 1000
Variable: All_Crimes
Density: 1000
Variable: All_Schools
Density: 1000
Variable: Hospitals
Density: 1000
No points in the radius
Variable: RailwayStations
Density: 1000
No points in the radius
Variable: FerryTerminals
Density: 1000
No points in the radius
Variable: BusStops
Density: 1000
Variable: ListedBuildings
Density: 1000
Variable: RoadAccidents
Density: 1000


In [ ]:
#Add distance to the nearest, don't include geometry lines
for k in nearest_var_dict:
    print(k)
    Postcodes[k + "_Distance_to_Nearest"] = kf.dist_to_nearest(Postcodes, nearest_var_dict[k], return_geom = False)

School
Hospital
RailwayStation
FerryTerminals
BusStops
ListedBuilding


In [ ]:
#Find the average value within the list of radii
for k in average_var_dict:
    for r in density_r_list:
        Postcodes["Average_" + k + "_" + str(r)] = kf.average_within_radius(Postcodes, average_var_dict[k][0], average_var_dict[k][1], r)

In [ ]:
Postcodes.columns

Index(['Postcode', 'geometry', 'LSOA', 'Name', 'LReg_Sales_In_LSOA',
       'LReg_AvCost_In_LSOA', 'UDPRNs', 'Crimes_per1000houses_Vehicle_crime',
       'Crimes_per1000houses_Shoplifting', 'Crimes_per1000houses_Other_crime',
       'Crimes_per1000houses_Public_order',
       'Crimes_per1000houses_Criminal_damage_and_arson',
       'Crimes_per1000houses_Burglary',
       'Crimes_per1000houses_Anti-social_behaviour',
       'Crimes_per1000houses_Theft_from_the_person',
       'Crimes_per1000houses_Drugs',
       'Crimes_per1000houses_Violence_and_sexual_offences',
       'Crimes_per1000houses_Other_theft',
       'LReg_AvCost_In_LSOA_within_1_boundaries',
       'LReg_AvCost_In_LSOA_within_3_boundaries',
       'LReg_AvCost_In_LSOA_within_5_boundaries',
       'Crimes_per1000houses_Vehicle_crime_within_1_boundaries',
       'Crimes_per1000houses_Vehicle_crime_within_2_boundaries',
       'Crimes_per1000houses_Vehicle_crime_within_5_boundaries',
       'Crimes_per1000houses_Shoplifting_w

In [ ]:
Postcodes.head()

,Postcode,geometry,LSOA,Name,LReg_Sales_In_LSOA,LReg_AvCost_In_LSOA,UDPRNs,Crimes_per1000houses_Vehicle_crime,Crimes_per1000houses_Shoplifting,Crimes_per1000houses_Other_crime,...,RailwayStation_Distance_to_Nearest,FerryTerminals_Distance_to_Nearest,BusStops_Distance_to_Nearest,ListedBuilding_Distance_to_Nearest,Average_ProertyPrice_2020_2021_1000,Average_TotalRainfall_mm_2020_1000,Average_Snow_Days_2020_1000,Average_Sunlight_h_2020_1000,Average_AverageTemperature_C_2020_1000,Average_GroundFrost_Days_2020_1000
0,LA120AP,POINT (325165.000 475838.000),Barrow-in-Furness 001C,Barrow-in-Furness 001C,14,217853.571429,344,0.0,0.0,0.0,...,-1,-1,55.009090,143.182118,229997.222222,1487.755635,0.543369,1695.706051,10.123339,102.051223
1,LA120LA,POINT (325297.000 476068.000),Barrow-in-Furness 001C,Barrow-in-Furness 001C,14,217853.571429,344,0.0,0.0,0.0,...,-1,-1,157.889202,321.036308,229997.222222,1487.755635,0.543369,1695.706051,10.123339,102.051223
2,LA120LB,POINT (325303.000 476138.000),Barrow-in-Furness 001C,Barrow-in-Furness 001C,14,217853.571429,344,0.0,0.0,0.0,...,-1,-1,220.429127,354.927703,229997.222222,1494.545881,0.583170,1693.049744,10.098081,102.744346
3,LA120LD,POINT (325163.000 475447.000),Barrow-in-Furness 001C,Barrow-in-Furness 001C,14,217853.571429,344,0.0,0.0,0.0,...,-1,-1,318.064459,385.869042,246871.875000,1486.433969,0.399297,1701.942061,10.210142,101.024964
4,LA120LF,POINT (324982.000 476371.000),Barrow-in-Furness 001C,Barrow-in-Furness 001C,14,217853.571429,344,0.0,0.0,0.0,...,-1,-1,588.891331,312.279806,229997.222222,1481.846499,0.599616,1694.205017,10.090729,102.042273
